# DICOM Example - Pancreas MRI
This notebook demonstrates how to process imaging data in DICOM format, ready to work with MONAI Label and 3D Slicer.

When launching the JupyterLab instance, make sure to select the **MONAI_ML** option in the Feature dropdown. 


For general examples of using Slicer within Jupyter notebook, please see: https://github.com/Slicer/SlicerNotebooks. For MONAI Label example notebooks and tutorials, see https://github.com/Project-MONAI/tutorials/tree/main/monailabel.


In [ ]:
import slicer, vtk
import os
import random
import dxpy
import JupyterNotebooksLib as slicernb
from MONAILabel import MONAILabelLogic
from DICOMLib import DICOMUtils

## 1. Prepare DICOM Data
Select **Slicer 5.8** kernel (On the top right of the notebook)

### Download data from Project to the Execution Environment
To process your data saved on the platform in the notebook, the data must be available in the execution environment (as is the case with any DNAnexus app). You can download input data from a project to your notebook using dx download (bash) or download_dxfile (python) via a notebook cell: 

In [ ]:
# specify field ID of interest
FIELD_ID = "20260"
PROJECT_ID = dxpy.WORKSPACE_ID  # Current project context

# find 10 files for field ID in current project
query = list(dxpy.find_data_objects(
    project=PROJECT_ID,
    classname="file",
    properties={"field_id": FIELD_ID},
    limit=10 
))

if not query:
    raise FileNotFoundError(f"No files found for field {FIELD_ID}")

# pick a random file 
random_file = random.choice(query)
file_id = random_file['id']

# download folder to instance
dxpy.download_dxfile(file_id, "example_data.zip")  
print("Downloaded and renamed file to: example_data.zip")

In [ ]:
DICOM_DIR = os.path.join(os.getcwd(), "example_data")
os.makedirs(DICOM_DIR, exist_ok=True)
os.environ["DICOM_DIR"] = DICOM_DIR

In [ ]:
! apt-get -qq update && apt-get install unzip
! unzip -qq example_data.zip -d $DICOM_DIR

### Option 1: Convert to NIfTI
This is often the simplest approach for local DICOM directories. For example, use a tool like `dcm2niix` to convert your DICOM series into single NIfTI volumes, which MONAI Label can easily load. Start the server pointing to the directory containing converted NIfTI files.

In [ ]:
! apt-get update && apt-get -y install dcm2niix

In [ ]:
! mkdir -p nifti_example_data
! dcm2niix -o nifti_example_data -m y $DICOM_DIR

### Option 2: Use the DICOM Module in 3D Slicer
Load your DICOM directory directly into the Slicer DICOM database and then either export entire DICOM database to research file format or use the MONAI Label module to upload the loaded volume to the server.

#### Load DICOM files into Slicer DICOM database

In [ ]:
# Switch to DICOM module
slicer.util.selectModule("DICOM")

# Instantiate a new DICOM browser
dicomBrowser = slicer.modules.DICOMWidget.browserWidget.dicomBrowser

# Create database directory
dicomBrowser.createNewDatabaseDirectory()

# Import directory containing DICOM files
dicomBrowser.importDirectory(DICOM_DIR)

# Wait for import to finish before proceeding (optional, if removed then import runs in the background)
dicomBrowser.waitForImportFinished()

print(f"Database stored at: {slicer.dicomDatabase.databaseDirectory}")

#### Explore loaded data in the Slicer DICOM database

In [ ]:
db = slicer.dicomDatabase

patientList = list(db.patients())
print(f"Patients: {patientList}")

studyList = list(db.studiesForPatient(patientList[0]))
print(f"Studies for patient {patientList[0]}: {studyList}")

seriesList = list(db.seriesForStudy(studyList[0]))
print(f"Series for study {studyList[0]}: {seriesList}")

fileList = db.filesForSeries(seriesList[0])
print(f"Number of files for series {seriesList[0]}: {len(fileList)}")

#### 2.a Export DICOM series from the database to research file format
Export the entire Slicer DICOM database content to nifti (or nrrd, etc.) file format with filtering of data type and naming of the output file based on DICOM tags. Start the server pointing to this directory.

In [ ]:
outputFolder = os.path.join(os.getcwd(), "nifti_example_data_2/")

from DICOMLib import DICOMUtils
patientUIDs = slicer.dicomDatabase.patients()
for patientUID in patientUIDs:
    loadedNodeIDs = DICOMUtils.loadPatientByUID(patientUID)
    for loadedNodeID in loadedNodeIDs:
        # Check if we want to save this node
        node = slicer.mrmlScene.GetNodeByID(loadedNodeID)
        # Only export images
        if not node or not node.IsA('vtkMRMLScalarVolumeNode'):
            continue
        # Construct filename
        shNode = slicer.mrmlScene.GetSubjectHierarchyNode()
        seriesItem = shNode.GetItemByDataNode(node)
        studyItem = shNode.GetItemParent(seriesItem)
        patientItem = shNode.GetItemParent(studyItem)
        filename = shNode.GetItemAttribute(patientItem, 'DICOM.PatientID')
        filename += '_' + shNode.GetItemAttribute(studyItem, 'DICOM.StudyDate')
        filename += '_' + shNode.GetItemAttribute(seriesItem, 'DICOM.SeriesNumber')
        filename += '_' + shNode.GetItemAttribute(seriesItem, 'DICOM.Modality')
        filename = slicer.app.ioManager().forceFileNameValidCharacters(filename) + ".nii"
        # Save node
        print(f'Write {node.GetName()} to {filename}')
        success = slicer.util.saveNode(node, outputFolder+filename)
    slicer.mrmlScene.Clear()

#### 2.b Use the MONAI Label module to upload the loaded volume to the server
Start MONAI Label server with --studies poiniting to an empty directory, where the volumes will be later uploaded.

##### Load specific series to the Slicer scene

In [ ]:
serieToLoad = seriesList[0]
print(f"Loading serie {serieToLoad} into the Slicer scene")

loadedNodeIDs = DICOMUtils.loadSeriesByUID([seriesList[0]])
print(f"Loaded node IDs: {loadedNodeIDs}")

##### Export loaded volume

In [ ]:
node = slicer.mrmlScene.GetNodeByID(loadedNodeIDs[0])
    
image_id = "image_1"
image_path = os.path.join(os.getcwd(), "image_1.nrrd")

slicer.util.saveNode(node, image_path)
print(f"Volume saved locally to: {image_path}")

After you start MONAI Label server with empty directory and connect from Slicer notebook, you can upload exported volume to the server:

```python
ml.upload_image(
    image_in=image_path,
    image_id=image_id
)
```


## 2. Start up the MONAI Label server with your model

Here is an example of how to start up MONAI label. When doing this in practice with the current pancreas data, you will need to use a custom model. Here we have substituted in the 'segmentation' MONAI model as a placeholder.

When the server is running, your terminal will now show the following message:

Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

Please do not disturb the terminal from this state whilst you are still working with the MONAI Label server. 


```bash

monailabel apps --download --name radiology --output apps
monailabel start_server --app apps/radiology --studies nifti_example_data_2 --conf models segmentation

```

### Connect to MONAI Label server

In [ ]:
SERVER_URL = "http://0.0.0.0:8000"

In [ ]:
ml = MONAILabelLogic()
ml.setServer(SERVER_URL)
ml.server_url

### Get server/app information

In [ ]:
ml.info()

### Query datastore (cases/images on the server)

In [ ]:
ml.datastore()